## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

In [160]:
import pandas as pd 
import numpy as np 


In [161]:
df = pd.read_csv('all_kindle_review.csv')

In [162]:
df= df[['reviewText','rating']]

In [163]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [164]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [165]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [166]:
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [167]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  12000 non-null  object
 1   rating      12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


In [169]:
import re 
from nltk.corpus import stopwords
import nltk
from bs4 import BeautifulSoup


In [170]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [171]:
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [172]:
def clean_text(text):
    ## delete HTML tags 
    text = BeautifulSoup(text,'html.parser').get_text()
    ## lower text 
    text = text.lower()
    ## delete url 
    text = re.sub(r"http\S+|www\S+", '', text)
    ## delete emails
    text = re.sub(r'\S+@\S+', '', text)
    ## delete punc
    text = re.sub(r'[^\w\s]', '', text)
    # إزالة التكرار الغلط زي "linethis" → line this
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    # حذف المسافات الزائدة
    text = re.sub(r'\s+', ' ', text).strip()
    ## delete numbers
    text = re.sub(r'\d+', '', text)
    ## Tokenize
    tokens = text.split()
    ## delete stopwords and lemmatization
    tokens = [lemmatizer.lemmatize(w, pos='v') for w in tokens if w not in stop_words and len(w) > 2]
    
    return ' '.join(tokens)

In [173]:
df['reviewText'] = df['reviewText'].apply(clean_text)

In [174]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df['reviewText'],df['rating'])

In [175]:
x_train

5365    plot stacy lonely decide make pass nash attrac...
9901    bob moats sucker punch vegas showgirl murder e...
5482    short story take flirt another level raunchy s...
416     book say south hot mess ideas potential never ...
2079               sure whether like lot steam care story
                              ...                        
7621    read review purchase first series shock negati...
7916    sex scenes prefer realistic interest one irrit...
5330    psychos meet must though bite disapointed end ...
5957    think might enjoy bite reviewers wasnt anythin...
9689    wellwritten fastpaced second chance romance se...
Name: reviewText, Length: 9000, dtype: object

## TF-IDF

In [176]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer(max_features= 3000,ngram_range=(1,2))

In [177]:
x_train_tfidf = tfidf_model.fit_transform(x_train).toarray()
x_test_tfidf = tfidf_model.transform(x_test).toarray()

In [178]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
logistic_model = LogisticRegression()
naive_model = MultinomialNB()


In [179]:
logistic_model.fit(x_train_tfidf,y_train)
naive_model.fit(x_train_tfidf,y_train)
y_pred_logistic = logistic_model.predict(x_test_tfidf)
y_pred_naive = naive_model.predict(x_test_tfidf)

In [180]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def evaluate (y_true , y_pred):
    accuracy = accuracy_score(y_true , y_pred)
    f1 = f1_score(y_true , y_pred)
    precision= precision_score(y_true , y_pred)
    recall= recall_score(y_true , y_pred)

    report = {
        'accuracy_score':accuracy,
        'f1_score':f1,
        'precision_score':precision,
        'recall_score':recall
    }
    return report

In [181]:
print('logistic') 
report = evaluate(y_test,y_pred_logistic)
print (report)

logistic
{'accuracy_score': 0.8356666666666667, 'f1_score': 0.8825910931174089, 'precision_score': 0.8403628117913832, 'recall_score': 0.9292878635907723}


In [182]:
print('naive') 
report = evaluate(y_test,y_pred_naive)
print (report)

naive
{'accuracy_score': 0.8186666666666667, 'f1_score': 0.8738989337042188, 'precision_score': 0.8125, 'recall_score': 0.9453360080240722}


## Word2Vec

In [186]:
corpus = df['reviewText'].tolist()

In [187]:
from nltk.tokenize import sent_tokenize

In [188]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(clean_text(sent).split())

In [189]:
words

[['jace',
  'rankin',
  'may',
  'short',
  'hes',
  'nothing',
  'mess',
  'man',
  'haul',
  'saloon',
  'undertaker',
  'know',
  'hes',
  'famous',
  'bounty',
  'hunter',
  'oregon',
  'shoot',
  'man',
  'saloon',
  'finish',
  'years',
  'long',
  'quest',
  'avenge',
  'sisters',
  'murder',
  'try',
  'figure',
  'next',
  'snottynosed',
  'farm',
  'boy',
  'rescue',
  'gang',
  'bully',
  'offer',
  'money',
  'kill',
  'man',
  'force',
  'ranch',
  'reluctantly',
  'agree',
  'bring',
  'man',
  'justice',
  'kill',
  'outright',
  'first',
  'need',
  'tell',
  'sisters',
  'widower',
  'newskyla',
  'kyle',
  'springer',
  'bailey',
  'rid',
  'trail',
  'sleep',
  'grind',
  'past',
  'month',
  'try',
  'find',
  'jace',
  'want',
  'revenge',
  'man',
  'kill',
  'husband',
  'take',
  'ranch',
  'amongst',
  'crimes',
  'shes',
  'keen',
  'detour',
  'jace',
  'want',
  'take',
  'realize',
  'shes',
  'options',
  'hide',
  'behind',
  'boy',
  'persona',
  'best',

In [190]:
import gensim

In [191]:
model=gensim.models.Word2Vec(words)

In [192]:
model.corpus_count

12000

In [193]:
model.wv.similar_by_word('good')

[('decent', 0.8609011769294739),
 ('okay', 0.8233892321586609),
 ('satisfy', 0.8099052906036377),
 ('line', 0.7930386066436768),
 ('bethe', 0.7870538830757141),
 ('description', 0.7853761315345764),
 ('totally', 0.785331130027771),
 ('enjoyable', 0.784841001033783),
 ('excite', 0.779928982257843),
 ('overall', 0.7788615822792053)]

In [194]:
model.wv['good'].shape

(100,)

In [195]:
!pip install tqdm 

In [196]:
from tqdm import tqdm

In [197]:
def avg_word2vec(doc, model, vector_size=100):
    vecs = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if vecs:
        return np.mean(vecs, axis=0)
    else:
        return np.zeros(vector_size)


In [198]:
X=[]
for i in tqdm(range(len(words))) :
    X.append(avg_word2vec(words[i],model))

100%|██████████| 12000/12000 [00:09<00:00, 1257.77it/s]


In [199]:
len(X)

12000

In [200]:
X_new=np.array(X)

In [201]:
X_new.shape

(12000, 100)

In [202]:
X_new[0].reshape(1,-1).shape

(1, 100)

In [203]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, df['rating'], test_size=0.2, random_state=42)

In [204]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)


LogisticRegression()

In [205]:
y_pred_log_word2vec = clf.predict(X_test)

In [206]:
print('logistc with word2vec')
report = evaluate(y_test,y_pred_log_word2vec)
print(report)

logistc with word2vec
{'accuracy_score': 0.7779166666666667, 'f1_score': 0.8434654919236417, 'precision_score': 0.7942477876106194, 'recall_score': 0.8991859737006888}


In [207]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier()
clf2.fit(X_train, y_train)
y_pred_random_word2vec = clf2.predict(X_test)

In [213]:
print('random with word2vec')
report = evaluate(y_test,y_pred_random_word2vec)
print(report)

random with word2vec
{'accuracy_score': 0.76375, 'f1_score': 0.8314005352363961, 'precision_score': 0.7916194790486977, 'recall_score': 0.8753913587977458}
